
<img src="img/viu_logo.png" width="200">

## 01MIAR - Procesamiento de Datos

![logo](img/python_logo.png)

*Ivan Fuertes*

# Sumario
- Trabajo con strings
- Combinando datasets
- Limpieza de datos
 - map, filter, reduce
 - filling missing values
 - valores duplicados
 - categorizacion de datos

In [1]:
import numpy as np
import pandas as pd

### Combinar varios datasets 
- En base a un elemento en común (índice)
- MovieLens 'UserId'

In [2]:
import zipfile as zp # para descomprimir archivos zip
import urllib.request # para descargar de URL
import os

# descargar MovieLens dataset
url = 'http://files.grouplens.org/datasets/movielens/ml-1m.zip'  
local_zip = os.path.join("res", "ml-1m.zip")
urllib.request.urlretrieve(url, local_zip)
# descomprimiendo archivo zip
with zp.ZipFile(local_zip, 'r') as zipp: 
    print('Extracting all files...') 
    zipp.extractall(os.path.join("res")) # destino
    print('Done!') 

Extracting all files...
Done!


In [3]:
root_path = os.path.join("res", "ml-1m" )

ratings_dataset = pd.read_csv(os.path.join(root_path, "ratings.dat"), sep='::',
                                index_col=0, engine='python', encoding="ISO-8859-1",
                                names=['UserID','MovieID','Rating','Timestamp'])

users_dataset = pd.read_csv(os.path.join(root_path, "users.dat"),sep='::',
                              index_col=0, engine='python',  encoding="ISO-8859-1",
                              names=['UserID','Gender','Age','Occupation','Zip-code'])

In [4]:
display(users_dataset.sample(5))

,Gender,Age,Occupation,Zip-code
UserID,,,,
40,M,45,0,10543
3304,F,45,5,92649
2173,M,50,13,87502
1640,M,35,17,95610
5537,F,56,1,10543


In [5]:
display(ratings_dataset.sample(5))

,MovieID,Rating,Timestamp
UserID,,,
3863,377,3,965856082
864,3075,5,977181523
4160,2377,4,965337380
2454,527,5,974181155
713,1101,5,975536106


### Uniendo datasets con 'join' y 'merge'
- merge() == join()
 - 'join' utiliza por defecto los índices para unir
- Utilizando el parámetro 'on'
 - Si las columnas difieren, 'left_on' y 'right_on'
 
 https://i.stack.imgur.com/hMKKt.jpg

In [6]:
# combinando users y ratings, ¿Cómo?
combined_dataset = users_dataset.merge(ratings_dataset, on='UserID', how='inner') # parametro 'on' define la columna pivote
display(combined_dataset.sample(5))
print(len(combined_dataset))

,Gender,Age,Occupation,Zip-code,MovieID,Rating,Timestamp
UserID,,,,,,,
1541,M,56,17,23464-3016,1876,4,974743340
4748,M,45,16,60302,141,4,963268977
4022,M,35,14,53403,1234,3,965522159
1084,M,1,19,03226,34,2,974946858
3962,F,56,0,98115,3751,1,965671055


1000209


In [7]:
movies_dataset = pd.read_csv(os.path.join(root_path, "movies.dat"),sep='::', engine='python', encoding="ISO-8859-1", names=['MovieID','Title','Genre'])
display(movies_dataset.sample(5))

,MovieID,Title,Genre
577,581,"Celluloid Closet, The (1995)",Documentary
1440,1467,Salut cousin! (1996),Comedy|Drama
1784,1853,"Alan Smithee Film: Burn Hollywood Burn, An (1997)",Comedy
2891,2960,Beefcake (1999),Drama
3320,3389,Let's Get Harry (1986),Action|Adventure


In [8]:
# combinando movies y el resto
all_dataset = combined_dataset.merge(movies_dataset,on='MovieID', how='inner')
display(all_dataset.sample(5))

,Gender,Age,Occupation,Zip-code,MovieID,Rating,Timestamp,Title,Genre
85479,M,25,4,26508,292,4,965309217,Outbreak (1995),Action|Drama|Thriller
62287,M,45,12,92122,2194,3,974684352,"Untouchables, The (1987)",Action|Crime|Drama
661040,M,45,6,92103,313,3,959443593,"Swan Princess, The (1994)",Animation|Children's
819764,M,35,4,98125,2303,5,976319970,Nashville (1975),Drama|Musical
934305,M,50,20,48230,2519,3,957459704,House on Haunted Hill (1958),Horror


### Concatenate
https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.concat.html

## Pivot
- Representar los datos en función a varios parámetros, agregando
```python
pivot_table(<lista de valores>, index=<agregador primario>, columns=<agregador secundario>)
```
- https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.pivot_table.html
- https://pandas.pydata.org/pandas-docs/stable/user_guide/reshaping.html

In [9]:
display(all_dataset.pivot_table('Rating', index='Gender', columns='Age'))
display(all_dataset.pivot_table('Rating', index='Gender', columns='Age', aggfunc='count'))
display(all_dataset.pivot_table('Rating', index='Gender', columns='Age', aggfunc=['count', 'mean']))

Age,1,18,25,35,45,50,56
Gender,,,,,,,
F,3.616291,3.453145,3.60670,3.659653,3.663044,3.797110,3.915534
M,3.517461,3.525476,3.52678,3.604434,3.627942,3.687098,3.720327


Age,1,18,25,35,45,50,56
Gender,,,,,,,
F,8827,45427,91340,49473,24110,18064,9199
M,18384,138109,304216,149530,59523,54426,29581


count                                                   mean  \
Age        1       18      25      35     45     50     56        1    
Gender                                                                 
F        8827   45427   91340   49473  24110  18064   9199  3.616291   
M       18384  138109  304216  149530  59523  54426  29581  3.517461   

                                                                   
Age           18       25        35        45        50        56  
Gender                                                             
F       3.453145  3.60670  3.659653  3.663044  3.797110  3.915534  
M       3.525476  3.52678  3.604434  3.627942  3.687098  3.720327

## Agrupaciones
- agg -> funciones estadísticas de agregación
- Series.unique() -> valores únicos
- pd.value_counts -> ocurrencias

## Manipulación de strings
```python
split(): separar en bloques en función de un carácter
replace(): reemplazar un carácter por otro
index(): encontrar la posición de un carácter
```

In [10]:
# Ejemplo con MovieLens: Genre
## 1: obtener todos los géneros por separado
## 2: crear un dataset de géneros
## 3: por película, marcar género por separado
## 4: unir con dataset original
display(movies_dataset.head(3))

,MovieID,Title,Genre
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance


In [12]:
all_genres = movies_dataset['Genre'].apply(lambda x : x.split('|'))
print(all_genres)

# print([genre for x in all_genres for genre in x])

genres = pd.unique([genre for x in all_genres for genre in x])
print(genres)

genres1 = pd.unique(all_genres.sum())
print(genres1)

0        [Animation, Children's, Comedy]
1       [Adventure, Children's, Fantasy]
2                      [Comedy, Romance]
3                        [Comedy, Drama]
4                               [Comedy]
                      ...               
3878                            [Comedy]
3879                             [Drama]
3880                             [Drama]
3881                             [Drama]
3882                   [Drama, Thriller]
Name: Genre, Length: 3883, dtype: object
['Animation' "Children's" 'Comedy' 'Adventure' 'Fantasy' 'Romance' 'Drama'
 'Action' 'Crime' 'Thriller' 'Horror' 'Sci-Fi' 'Documentary' 'War'
 'Musical' 'Mystery' 'Film-Noir' 'Western']
['Animation' "Children's" 'Comedy' 'Adventure' 'Fantasy' 'Romance' 'Drama'
 'Action' 'Crime' 'Thriller' 'Horror' 'Sci-Fi' 'Documentary' 'War'
 'Musical' 'Mystery' 'Film-Noir' 'Western']


In [13]:
# crear tabla con columnas por género
zeros = np.zeros( (len(movies_dataset), len(genres)) )
genres_frame = pd.DataFrame(zeros, columns=genres)
display(genres_frame.head(3))

,Animation,Children's,Comedy,Adventure,Fantasy,Romance,Drama,Action,Crime,Thriller,Horror,Sci-Fi,Documentary,War,Musical,Mystery,Film-Noir,Western
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [14]:
columns_genres = genres_frame.columns # lista de generos (columnas)
# para cada película, marcar género con 1
for i, genre in enumerate(movies_dataset['Genre']):
    inds = columns_genres.get_indexer(genre.split('|')) # retorna los indices correspondientes a los generos de cada pelicula
    genres_frame.iloc[i,inds] = 1 # localiza las columnas del genero correspondiente, marca con 1

In [15]:
display(genres_frame.head(5))

,Animation,Children's,Comedy,Adventure,Fantasy,Romance,Drama,Action,Crime,Thriller,Horror,Sci-Fi,Documentary,War,Musical,Mystery,Film-Noir,Western
0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [16]:
# unir con dataset original
movies_split_genre = movies_dataset.join(genres_frame)

In [17]:
display(movies_split_genre.head(5))

,MovieID,Title,Genre,Animation,Children's,Comedy,Adventure,Fantasy,Romance,Drama,...,Crime,Thriller,Horror,Sci-Fi,Documentary,War,Musical,Mystery,Film-Noir,Western
0,1,Toy Story (1995),Animation|Children's|Comedy,1.0,1.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2,Jumanji (1995),Adventure|Children's|Fantasy,0.0,1.0,0.0,1.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,3,Grumpier Old Men (1995),Comedy|Romance,0.0,0.0,1.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,4,Waiting to Exhale (1995),Comedy|Drama,0.0,0.0,1.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,5,Father of the Bride Part II (1995),Comedy,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


#### Replace e index para extraer el año de la película

In [18]:
display(movies_dataset.head(2))

,MovieID,Title,Genre
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy


In [19]:
# extraer el año de la columna Title
def split_year(title):
    index = title.index('(')
    return title[index:].replace('(','').replace(')','')
    
# crear nueva columna Year
movies_dataset['Year'] = movies_dataset['Title'].apply(split_year)
display(movies_dataset.sample(2))


,MovieID,Title,Genre,Year
457,461,Go Fish (1994),Drama|Romance,1994
1405,1429,Jackie Chan's First Strike (1996),Action,1996


In [20]:
# eliminar el año de la columna Title
def remove_year(title):
    index = title.index('(')
    return title[:index-1].strip()

movies_dataset['Title'] = movies_dataset['Title'].apply(remove_year)
display(movies_dataset.head(2))

,MovieID,Title,Genre,Year
0,1,Toy Story,Animation|Children's|Comedy,1995
1,2,Jumanji,Adventure|Children's|Fantasy,1995


#### Expresiones regulares
https://docs.python.org/3/library/re.html

- import re

In [21]:
# ¿Cómo localizar que 'Zip-code' tiene un formato erróneo?
users_dataset.sample(5)

# users_dataset['Zip-code'].str.match('^\d{5}$')

display(users_dataset[users_dataset['Zip-code'].str.match('^\d{5}$') == False])

# ^\d{5}$
# ^ = start of the string
# \d = decimal string
# {5} = 5 repeticiones de decimales
# $ = end of string

,Gender,Age,Occupation,Zip-code
UserID,,,,
161,M,45,16,98107-2117
233,F,45,20,37919-4204
293,M,56,1,55337-4056
458,M,50,16,55405-2546
506,M,25,16,55103-1006
...,...,...,...,...
5682,M,18,0,23455-4959
5904,F,45,12,954025
5925,F,25,0,90035-4444


In [22]:
# ¿Cómo extraer el año con regex en el formato adecuado?
movies_dataset = pd.read_csv(os.path.join(root_path, "movies.dat"),sep='::', engine='python', encoding='ISO-8859-1', names=['MovieID','Title','Genre'])
display(movies_dataset.head(2))
display(movies_dataset['Title'].str.extract('(\d{4})'))

# (\d{4})
# (= busca apertura parentesis
# \d = decimal string
# {4} = 4 repeticiones de decimales
# ) = cierre de parentesis

,MovieID,Title,Genre
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy


,0
0,1995
1,1995
2,1995
3,1995
4,1995
...,...
3878,2000
3879,2000
3880,2000
3881,2000


## Operaciones con colecciones
```python
reduce: aplicar una operación y retornar un valor
map: aplicar  una operación y retornar una secuencia
filter: retorna una secuencia con elementos que cumplen una condición
```


## Reduce
- Aplicar una operación matemática a cada uno de los elementos de una colección
- Diferente de 'apply()' porque retorna un valor numérico
- Ejemplo: Detección de géneros en años específicos

https://docs.python.org/3/library/functools.html

In [23]:
from functools import reduce # necesario para reduce

lista = [1, 3, 5, 7, 9]
print(reduce(lambda x,y: x + y, lista))

25


In [24]:
movies_1975 = movies_split_genre[ movies_split_genre['Title'].str.contains('1975') ]
movies_1975.head(3)

,MovieID,Title,Genre,Animation,Children's,Comedy,Adventure,Fantasy,Romance,Drama,...,Crime,Thriller,Horror,Sci-Fi,Documentary,War,Musical,Mystery,Film-Noir,Western
707,716,Switchblade Sisters (1975),Crime,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
994,1007,"Apple Dumpling Gang, The (1975)",Children's|Comedy|Western,0.0,1.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
996,1009,Escape to Witch Mountain (1975),Adventure|Children's|Fantasy,0.0,1.0,0.0,1.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [25]:
any_drama = reduce(lambda x,y : bool(x) | bool(y),movies_1975['Drama']) # hay algún drama en 1975
print(any_drama)

all_comedy = reduce(lambda x,y : bool(x) & bool(y),movies_1975['Comedy']) # son todas las películas de 1975 comedias?
print(all_comedy)

True
False


In [26]:
print(movies_1975['Drama'].any()) # Comprueba si hay algún valor que puede cumplir  
print(movies_1975['Comedy'].all()) # Comprueba si todos los valores son True

True
False


In [27]:
# Observar el tipo de dato antes para ver si es posible aplicar las funciones
print(movies_1975.dtypes)
print(movies_1975['Comedy'].unique())

MovieID          int64
Title           object
Genre           object
Animation      float64
Children's     float64
Comedy         float64
Adventure      float64
Fantasy        float64
Romance        float64
Drama          float64
Action         float64
Crime          float64
Thriller       float64
Horror         float64
Sci-Fi         float64
Documentary    float64
War            float64
Musical        float64
Mystery        float64
Film-Noir      float64
Western        float64
dtype: object
[0. 1.]


## Filter
- retorna una secuencia con elementos que cumplen una condición
- Ejemplo: obtener las películas de 1975 que contienen 'The' en el título

In [28]:
filtro = filter(lambda x : 'The' in x, movies_1975['Title'])
list(filtro)
# ¿Están todos los títulos con "The"? si tiene mayúsculas o no...

['Apple Dumpling Gang, The (1975)',
 'Man Who Would Be King, The (1975)',
 'Stepford Wives, The (1975)',
 'Rocky Horror Picture Show, The (1975)',
 'McCullochs, The (1975)',
 'Mirror, The (Zerkalo) (1975)']

## Map
- aplicar  una operación y retornar una secuencia
- Cambiar el valor integral de la columna 'Comedy' por bool

In [29]:
mapa = map(lambda x : bool(x), movies_split_genre['Comedy'])
movies_split_genre.loc[:,'Comedy'] = list(mapa)
display(movies_split_genre.head(4))

,MovieID,Title,Genre,Animation,Children's,Comedy,Adventure,Fantasy,Romance,Drama,...,Crime,Thriller,Horror,Sci-Fi,Documentary,War,Musical,Mystery,Film-Noir,Western
0,1,Toy Story (1995),Animation|Children's|Comedy,1.0,1.0,True,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2,Jumanji (1995),Adventure|Children's|Fantasy,0.0,1.0,False,1.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,3,Grumpier Old Men (1995),Comedy|Romance,0.0,0.0,True,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,4,Waiting to Exhale (1995),Comedy|Drama,0.0,0.0,True,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


## Transformación de variables (calidad de datos)
- Valores no definidos
- Valores duplicados
- Discretización (valores categóricos)

In [30]:
matrix = pd.DataFrame(np.random.randint(10,size=(5,10)))
matrix[matrix < 2] = np.nan
display(matrix)

,0,1,2,3,4,5,6,7,8,9
0,4.0,3.0,9.0,8.0,7,4.0,5,NaN,8,6.0
1,6.0,4.0,NaN,2.0,2,5.0,5,NaN,2,NaN
2,4.0,NaN,8.0,6.0,7,9.0,3,4.0,8,9.0
3,NaN,3.0,5.0,9.0,8,2.0,8,2.0,3,6.0
4,8.0,6.0,4.0,NaN,6,NaN,2,3.0,3,2.0


In [31]:
# nulos por columna
print(matrix.isnull().sum())
# display(matrix.isna().sum())

0    1
1    1
2    1
3    1
4    0
5    1
6    0
7    2
8    0
9    1
dtype: int64


In [32]:
# Cantidad valores nulos
print(matrix.isnull().sum().sum())

8


In [33]:
# numero de no nulos por fila
print(matrix.count(axis=1))

0    9
1    7
2    9
3    9
4    8
dtype: int64


In [34]:
# Número de nulos por fila
print(matrix.shape[1] - matrix.count(axis=1))

0    1
1    3
2    1
3    1
4    2
dtype: int64


In [35]:
# Representación de las filas en las que una determinada columna tiene nulos
display(matrix[matrix[1].isnull()])

,0,1,2,3,4,5,6,7,8,9
2,4.0,NaN,8.0,6.0,7,9.0,3,4.0,8,9.0


In [36]:
# Conteo de valores que aparecen en el dataset
valores = [8, 4]
# Identificación de valores de dominio que se encuentran en un listado
display(matrix[matrix[6].isin(valores)])

,0,1,2,3,4,5,6,7,8,9
3,NaN,3.0,5.0,9.0,8,2.0,8,2.0,3,6.0


In [37]:
## Tratamiento de valores nulos
# eliminar
display(matrix.dropna())

,0,1,2,3,4,5,6,7,8,9


In [38]:
# eliminar si no hay un número de valores no NaN
display(matrix.dropna(thresh=7))

,0,1,2,3,4,5,6,7,8,9
0,4.0,3.0,9.0,8.0,7,4.0,5,NaN,8,6.0
1,6.0,4.0,NaN,2.0,2,5.0,5,NaN,2,NaN
2,4.0,NaN,8.0,6.0,7,9.0,3,4.0,8,9.0
3,NaN,3.0,5.0,9.0,8,2.0,8,2.0,3,6.0
4,8.0,6.0,4.0,NaN,6,NaN,2,3.0,3,2.0


In [39]:
# sustituir por un valor fijo
display(matrix.fillna(-1))

,0,1,2,3,4,5,6,7,8,9
0,4.0,3.0,9.0,8.0,7,4.0,5,-1.0,8,6.0
1,6.0,4.0,-1.0,2.0,2,5.0,5,-1.0,2,-1.0
2,4.0,-1.0,8.0,6.0,7,9.0,3,4.0,8,9.0
3,-1.0,3.0,5.0,9.0,8,2.0,8,2.0,3,6.0
4,8.0,6.0,4.0,-1.0,6,-1.0,2,3.0,3,2.0


In [40]:
# sustituir por valor dinámico (copia)
display(matrix)
display(matrix.fillna(method='bfill')) # bfill y ffill

,0,1,2,3,4,5,6,7,8,9
0,4.0,3.0,9.0,8.0,7,4.0,5,NaN,8,6.0
1,6.0,4.0,NaN,2.0,2,5.0,5,NaN,2,NaN
2,4.0,NaN,8.0,6.0,7,9.0,3,4.0,8,9.0
3,NaN,3.0,5.0,9.0,8,2.0,8,2.0,3,6.0
4,8.0,6.0,4.0,NaN,6,NaN,2,3.0,3,2.0


,0,1,2,3,4,5,6,7,8,9
0,4.0,3.0,9.0,8.0,7,4.0,5,4.0,8,6.0
1,6.0,4.0,8.0,2.0,2,5.0,5,4.0,2,9.0
2,4.0,3.0,8.0,6.0,7,9.0,3,4.0,8,9.0
3,8.0,3.0,5.0,9.0,8,2.0,8,2.0,3,6.0
4,8.0,6.0,4.0,NaN,6,NaN,2,3.0,3,2.0


In [41]:
# sustituir por valor dinámico (interpolación)
display(matrix)
display(matrix.interpolate())

,0,1,2,3,4,5,6,7,8,9
0,4.0,3.0,9.0,8.0,7,4.0,5,NaN,8,6.0
1,6.0,4.0,NaN,2.0,2,5.0,5,NaN,2,NaN
2,4.0,NaN,8.0,6.0,7,9.0,3,4.0,8,9.0
3,NaN,3.0,5.0,9.0,8,2.0,8,2.0,3,6.0
4,8.0,6.0,4.0,NaN,6,NaN,2,3.0,3,2.0


,0,1,2,3,4,5,6,7,8,9
0,4.0,3.0,9.0,8.0,7,4.0,5,NaN,8,6.0
1,6.0,4.0,8.5,2.0,2,5.0,5,NaN,2,7.5
2,4.0,3.5,8.0,6.0,7,9.0,3,4.0,8,9.0
3,6.0,3.0,5.0,9.0,8,2.0,8,2.0,3,6.0
4,8.0,6.0,4.0,9.0,6,2.0,2,3.0,3,2.0


#### Tratar valores duplicados

In [42]:
serie = pd.Series(['a','b','c','a','c','a','g'])
print(serie.duplicated())

0    False
1    False
2    False
3     True
4     True
5     True
6    False
dtype: bool


In [43]:
df = all_dataset
display(df.head(3))

# Eliminación de los duplicados en una columna definida
df2 = df.drop_duplicates(subset="Gender", keep='last', inplace=False)
display(df2)

,Gender,Age,Occupation,Zip-code,MovieID,Rating,Timestamp,Title,Genre
0,F,1,10,48067,1193,5,978300760,One Flew Over the Cuckoo's Nest (1975),Drama
1,M,56,16,70072,1193,5,978298413,One Flew Over the Cuckoo's Nest (1975),Drama
2,M,25,12,32793,1193,4,978220179,One Flew Over the Cuckoo's Nest (1975),Drama


,Gender,Age,Occupation,Zip-code,MovieID,Rating,Timestamp,Title,Genre
1000207,F,18,20,55410,3607,5,957756608,One Little Indian (1973),Comedy|Drama|Western
1000208,M,25,1,35401,2909,4,957273353,"Five Wives, Three Secretaries and Me (1998)",Documentary


#### Discretización (valores categóricos)
- Tras Series y DataFrame, objeto para categorías: Categorical
```python
categorias = pd.cut(<valores>, <bins>) 
```

In [44]:
# especificar los bloques
bins = [0,18,35,65,99]
edades = [16,25,18,71,44,100,12]
categorias = pd.cut(edades,bins)
print(categorias)

[(0.0, 18.0], (18.0, 35.0], (0.0, 18.0], (65.0, 99.0], (35.0, 65.0], NaN, (0.0, 18.0]]
Categories (4, interval[int64]): [(0, 18] < (18, 35] < (35, 65] < (65, 99]]


In [45]:
categorias.value_counts()

(0, 18]     3
(18, 35]    1
(35, 65]    1
(65, 99]    1
dtype: int64

In [46]:
# especificar el número de bloques
bins = 5
edades = [0,6,8,16,25,18,71,44,100]
categorias = pd.cut(edades,bins) # rangos idénticos (similar distancia de rangos)
print(categorias)
print(categorias.value_counts())

[(-0.1, 20.0], (-0.1, 20.0], (-0.1, 20.0], (-0.1, 20.0], (20.0, 40.0], (-0.1, 20.0], (60.0, 80.0], (40.0, 60.0], (80.0, 100.0]]
Categories (5, interval[float64]): [(-0.1, 20.0] < (20.0, 40.0] < (40.0, 60.0] < (60.0, 80.0] < (80.0, 100.0]]
(-0.1, 20.0]     5
(20.0, 40.0]     1
(40.0, 60.0]     1
(60.0, 80.0]     1
(80.0, 100.0]    1
dtype: int64


In [47]:
bins = 5
edades = [1,6,8,16,25,18,71,44,100]
categorias = pd.qcut(edades,bins) # rangos homogéneos (similar número de valores)
print(categorias)
print(categorias.value_counts())

[(0.999, 7.2], (0.999, 7.2], (7.2, 16.4], (7.2, 16.4], (23.6, 54.8], (16.4, 23.6], (54.8, 100.0], (23.6, 54.8], (54.8, 100.0]]
Categories (5, interval[float64]): [(0.999, 7.2] < (7.2, 16.4] < (16.4, 23.6] < (23.6, 54.8] < (54.8, 100.0]]
(0.999, 7.2]     2
(7.2, 16.4]      2
(16.4, 23.6]     1
(23.6, 54.8]     2
(54.8, 100.0]    2
dtype: int64
